In [5]:
# import sys
# !C:\Users\Alex\anaconda3\envs\tfenv\python.exe -m pip install torch

   ---------------------------------------- 111.0/111.0 MB 10.3 MB/s  0:00:10
   ---------------------------------------- 1.7/1.7 MB 10.4 MB/s  0:00:00
   ---------------------------------------- 6.3/6.3 MB 11.0 MB/s  0:00:00
   ---------------------------------------- 536.2/536.2 kB 8.8 MB/s  0:00:00



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
# Imports and model loading

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

C:\Users\Alex\anaconda3\envs\tfenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cpu'

In [2]:
model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


C:\Users\Alex\anaconda3\envs\tfenv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alex\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
# Context: portion of one book of the Bible (Psalm 23, KJV)

psalm_23_context = """
The LORD is my shepherd; I shall not want.
He maketh me to lie down in green pastures: 
he leadeth me beside the still waters.
He restoreth my soul: he leadeth me in the paths of righteousness for his name's sake.
Yea, though I walk through the valley of the shadow of death, 
I will fear no evil: for thou art with me; thy rod and thy staff they comfort me.
Thou preparest a table before me in the presence of mine enemies: 
thou anointest my head with oil; my cup runneth over.
Surely goodness and mercy shall follow me all the days of my life: 
and I will dwell in the house of the LORD for ever.
"""

In [13]:
# Helper function for generative Q&A

def answer_question(
    context: str,
    question: str,
    temperature: float,
    max_new_tokens: int,
    top_p: float,
):
    """
    Generate an answer to `question` based on `context`
    using google/flan-t5-small.
    """
    prompt = (
        "You are a helpful assistant that answers questions based only on the context.\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\n"
        "Answer in one or two sentences."
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,          # needed for temperature & top_p to matter
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id,
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded


In [14]:
# Parameter Experimentation (Temperature)

question = "According to the passage, why does the speaker say he will fear no evil?"

In [102]:
# Test 1 – Low temperature (0.2)

temp_1 = 0.2
answer_temp_1 = answer_question(
    psalm_23_context,
    question,
    temperature=temp_1,
    max_new_tokens=50,
    top_p=0.5,
)

print(f"TEMPERATURE = {temp_1}")
print(answer_temp_1)

TEMPERATURE = 0.2
he walk through the valley of the shadow of death


In [108]:
# Test 2 – Medium temperature (0.7)

temp_2 = 0.5
answer_temp_2 = answer_question(
    psalm_23_context,
    question,
    temperature=temp_2,
    max_new_tokens=100,
    top_p=0.7,
)

print(f"TEMPERATURE = {temp_2}")
print(answer_temp_2)

TEMPERATURE = 0.5
he walk through the valley of the shadow of death


In [124]:
# Test 3 – High temperature (1.0)

temp_3 = 1.0
answer_temp_3 = answer_question(
    psalm_23_context,
    question,
    temperature=temp_3,
    max_new_tokens=30,
    top_p=1,
)

print(f"TEMPERATURE = {temp_3}")
print(answer_temp_3)
     
      

TEMPERATURE = 1.0
for thou art with me.


In [135]:
# Final “best” run (combination of parameters)

best_temperature = 1
best_top_p = 0.9
best_max_new_tokens = 50

best_answer = answer_question(
    psalm_23_context,
    question,
    temperature=best_temperature,
    max_new_tokens=best_max_new_tokens,
    top_p=best_top_p,
)

print("FINAL BEST SETTINGS:")
print(f"temperature = {best_temperature}, top_p = {best_top_p}, max_new_tokens = {best_max_new_tokens}")
print(best_answer)


FINAL BEST SETTINGS:
temperature = 1, top_p = 0.9, max_new_tokens = 50
thou art with me; thy rod and thy staff they comfort me
